### Import and Clean Data Script

#### Author: Lauren Thomas
#### Created: 01/05/2021
#### Last updated: 20/07/2021

###### File description: This file imports, cleans and pre-processes the data that will be used in the ML models.

In [1]:
import os
import gzip
import glob
import json
import pickle

import geopandas as gp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import osmium as osm
from os import sep
from shapely.geometry import Point,Polygon,MultiPolygon

C:\Users\ltswe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ltswe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ltswe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Working directory
cwd = f"C:{sep}Users{sep}ltswe{sep}Dropbox{sep}Oxford{sep}Thesis"
# Data directory is kept on flash
data_dir = "D:"

#### Assign Census Tracts using Shapefiles

In [3]:
# Read in NYC Census Tract files
nycct = gp.read_file(f'{cwd}{sep}data{sep}shapefiles{sep}nyct2010.shp')

In [4]:
# Project shapefile into lat/long
nycct_proj = nycct.to_crs("epsg:4326")

# Generate full geoids (state FIPS + county FIPS + CT FIPS)
# NY State FIPS: 36
# County fips: 061 (Manhattan), 005 (Bronx), 081 (Queens), 085 (Staten Island), 047 (Brooklyn).
# BoroCT2010 uses a weird county code system where 1=Manhattan,2=Bronx,3=Brooklyn,4=Queens,& 5=SI
def gen_fips(x):
    ''' x = BoroCT2010 code'''
    county_code = x[0:1]
    ct_code = x[1:7]
    if county_code == '5': #SI
        return '36'+'085' + ct_code
    elif county_code == '1': #Manhattan
        return '36'+'061' + ct_code
    elif county_code == '4': #Queens
        return '36'+'081' + ct_code
    elif county_code == '3': #Brooklyn
        return '36'+'047' + ct_code
    else: #Bronx
        return '36'+'005' +ct_code

nycct_proj['fips_code'] = nycct_proj['BoroCT2010'].apply(lambda x: gen_fips(x))

In [ ]:
nycct_proj.plot()

In [ ]:
# Create a function that rounds lat/long in df to six digits (what Twitter provides for lat/long)
def round_lat_long(df):
    df['latitude'] = df['latitude'].apply(lambda x: round(float(x), 6))
    df['longitude'] = df['longitude'].apply(lambda x: round(float(x), 6))

In [ ]:
# Create function that assigns census tract using R-tree spatial indices 
def assign_ct(geo_df, df_name_str, count_thousand=True,count_hundred=False):
    ''' 
    This function assigns a census tract to each observation in a GeoDataFrame using the nycct file created above. 
    Input: geo_df: This is a GeoDataFrame that contains a col entitled 'geometry' that has lat/long points
    df_name: This is a string that contains the name of the dataframe 
    count_thousand: If true, tells you when every 1000 census tracts have been assigned
    count_hundred: If true, tells you when every 100 census tracts have been assigned
    Output: The original GeoDataFrame with a new col entitled 'LocationCT' that has the census tract corresponding
    to each observation. 
    '''
    print("Assigning Census Tracts for DataFrame", df_name_str)
    
    # Begin by creating a spatial index for the R-trees for the Geodataframe
    df_sindex = geo_df.sindex
    print(f'Number of groups for dataframe {df_name_str}: ', len(df_sindex.leaves()), '\n')
    
    # Create a list of census tracts to iterate through
    tracts = list(nycct_proj.fips_code)


    # Iterate through tracts & find all the tweets that correspond to each given tract
    i = 1
    for tract in tracts:
        select_tract = nycct_proj.loc[nycct_proj['fips_code']==tract]
    
        # Get the bounding box coordinates of the census tract as a list
        bounds = list(select_tract.bounds.values[0])
    
        #Get the indices of the points that are inside the bounding box of the given census tract
        df_candidate_idx = list(df_sindex.intersection(bounds))
        df_candidates = geo_df.loc[df_candidate_idx]

        # Now make the precise Point in Polygon query
        df_final_selection = df_candidates.loc[df_candidates.intersects(select_tract['geometry'].values[0])]
    
        # Put correct tract back in original DF
        geo_df.loc[list(df_final_selection.index.values),'LocationCT'] = tract
    
        i+= 1
        if count_thousand == True:
            if i%1000 == 0:
                print(f'another thousand done! up to {i} census tracts for DataFrame {df_name_str}')
        elif count_hundred == True:
            if i%100 == 0:
                print(f'another hundred done! up to {i} census tracts for DataFrame {df_name_str}')
    
    # Drop any tweets that are outside NYC
    geo_df = geo_df[geo_df['LocationCT'].isna() == False].reset_index(drop=True)
    
    # Pickle dataframe
    df_pickle = open(f"{data_dir}{sep}pickle{sep}{df_name_str}.pickle", "wb")
    pickle.dump(geo_df, df_pickle)
    
    print(f'DataFrame {df_name_str} is all pickled and ready to go! \n')
    return geo_df
    

#### Import & preprocess Tweet data

In [ ]:
# Restructure JSONs into JSOLs (where each line = one tweet) for each month-year from Jan 2011 to Dec 2013
# make list of month-year pairs
ym_list = [str(year)+"-"+("0"+str(month))[-2:] for month in range(1,13) for year in range(2011,2014)]

# Create a dictionary that will contain all the JSONs for a given list of month-year pairs

def create_json(ym_list, json_pickle_str):
    all_jsons = dict()
    for ym in ym_list:
        print(ym)
        # Create a list of the jsons that fall into that y-m - excluding all outputs that ends in 00000.json.
        json_list = [j for j in glob.glob(f'{data_dir}{sep}raw_tweets{sep}{ym}*{sep}*.json', recursive=True) 
                 if j[-10:] != '00000.json']
        # Create list of JSONs that we will append to the larger dictionary 
        temp_json_list = list()
        for j in json_list:
            temp_json = json.load(open(j, encoding = 'utf-8'))['data']
            temp_json_list.extend(temp_json)
        # Add temp_dict to larger dictionary of all JSONs with the key as the year-month
        all_jsons[ym] = temp_json_list
    # Pickle JSON
    tweets_json_pickle = open(f"{data_dir}{sep}pickle{sep}{json_pickle_str}.pickle", "wb")
    pickle.dump(all_jsons, tweets_json_pickle)
    print(json_pickle_str, "pickled!")


In [ ]:
# Create JSONs in 12 chunks (one for each month; 3 years in total)
# json_01 = create_json(ym_list[0:3], 'json_01')
# json_02 = create_json(ym_list[3:6], 'json_02')
# json_03 = create_json(ym_list[6:9], 'json_03')
# json_04 = create_json(ym_list[9:12], 'json_04')
# json_05 = create_json(ym_list[12:15], 'json_05')
# json_06 = create_json(ym_list[15:18], 'json_06')
# json_07 = create_json(ym_list[18:21], 'json_07')
# json_08 = create_json(ym_list[21:24], 'json_08')
# json_09 = create_json(ym_list[24:27], 'json_09')
# json_10 = create_json(ym_list[27:30], 'json_10')
# json_11 = create_json(ym_list[30:33], 'json_11')
# json_12 = create_json(ym_list[33:36], 'json_12')

In [ ]:
def unpickle_json(num):
    ''' This function unpickles the relevant JSON'''
    return pickle.load(open(f"{data_dir}{sep}pickle{sep}json_{num}.pickle", "rb"))

In [ ]:
def create_tweet_gdf(json_num, json_ym_list): 
    '''
    This function creates a GeoDataFrame of the tweets in a given JSON with the important info from the JSON
    Input: json_num: month number of relevant JSON (in string form, like "01" for Jan, "02" for Feb, etc.)
    json_ym_list: extract of ym_list that contains the relevant month-year pairs (e.g. ym_list[0:7] for Jan, etc.)
    Output: GeoDataFrame of tweets in JSON that contain geo-coordinates
    '''
    print("Creating GeoDataFrame", json_num)
    json = unpickle_json(json_num)

    x_list, y_list, date_list,geo_list, author_list, tweet_list, lang_list, like_list, quote_list, reply_list, retweet_list, \
    text_list, ym_list = list(),list(),list(),list(),list(),list(),list(),list(),list(),list(),list(),list(),list()
    # Make lists to form columns in GeoDataFrame 
    for ym in json_ym_list:
        print(ym)
        for j in json[ym]:
            # try/except: if KeyError occurs, it means it doesn't have geo-coordinates, so we ignore those
            try:
                geo_id = Point(j["geo"]["coordinates"]["coordinates"])
                # Annoyingly, some of the tweets are geotagged outside NYC. Get rid of these.
                # These lat/long maxes and mins are taken from the NYC census tract shapefiles. 
                # Note x is long and y is lat (thanks, Twitter!)
                if geo_id.x <= -73.70000924132164 and geo_id.x >= -74.25559213002796 \
                and geo_id.y <= 40.91553243056209 and geo_id.y >= 40.49611511946593: 
                    date_list.append(j['created_at'])
                    x_list.append(geo_id.x)
                    y_list.append(geo_id.y)
                    geo_list.append(geo_id), author_list.append(j['author_id']), tweet_list.append(j['id'])
                    lang_list.append(j['lang']), like_list.append(j['public_metrics']['like_count'])
                    quote_list.append(j['public_metrics']['quote_count']), reply_list.append(j['public_metrics']['reply_count']) 
                    retweet_list.append(j['public_metrics']['retweet_count']), text_list.append(j['text'])
                    ym_list.append(ym)
            except KeyError:
                continue
    # Create geodataframe for given json (month) 
    tweet_df = gp.GeoDataFrame(
        {'ym': ym_list,
         'date':date_list,
        'tweet_id':tweet_list,
         'author_id':author_list,
        'lang': lang_list,
        'like_count': like_list,
         'quote_count': quote_list,
         'reply_count': reply_list,
         'retweet_count': retweet_list,
         'text': text_list,
         'x': x_list,
         'y': y_list,
         'geometry': geo_list
        }
    )
    # check for duplicates and drop any
    tweet_df = tweet_df.drop_duplicates()
    print("\n")
    return tweet_df

In [ ]:
# Use the create_gdf function to create GeoDataFrames for all the JSONs
tweet_gdf_01 = create_tweet_gdf('01',ym_list[0:3])
tweet_gdf_02 = create_tweet_gdf('02',ym_list[3:6])
tweet_gdf_03 = create_tweet_gdf("03", ym_list[6:9])
tweet_gdf_04 = create_tweet_gdf('04',ym_list[9:12])
tweet_gdf_05 = create_tweet_gdf('05',ym_list[12:15])
tweet_gdf_06 = create_tweet_gdf('06',ym_list[15:18])
tweet_gdf_07 = create_tweet_gdf('07',ym_list[18:21])
tweet_gdf_08 = create_tweet_gdf('08',ym_list[21:24])
tweet_gdf_09 = create_tweet_gdf('09',ym_list[24:27])
tweet_gdf_10 = create_tweet_gdf('10',ym_list[27:30])
tweet_gdf_11 = create_tweet_gdf('11',ym_list[30:33])
tweet_gdf_12 = create_tweet_gdf('12',ym_list[33:36])

In [ ]:
# Use the assign_ct function to assign census tracts to obs in each of the geodataframes
tweet_gdf_01 = assign_ct(tweet_gdf_01, 'tweet_gdf_01')
tweet_gdf_02 = assign_ct(tweet_gdf_02, 'tweet_gdf_02')
tweet_gdf_03 = assign_ct(tweet_gdf_03, 'tweet_gdf_03')
tweet_gdf_04 = assign_ct(tweet_gdf_04, 'tweet_gdf_04')
tweet_gdf_05 = assign_ct(tweet_gdf_05, 'tweet_gdf_05')
tweet_gdf_06 = assign_ct(tweet_gdf_06, 'tweet_gdf_06')
tweet_gdf_07 = assign_ct(tweet_gdf_07, 'tweet_gdf_07')
tweet_gdf_08 = assign_ct(tweet_gdf_08, 'tweet_gdf_08')
tweet_gdf_09 = assign_ct(tweet_gdf_09, 'tweet_gdf_09')
tweet_gdf_10 = assign_ct(tweet_gdf_10, 'tweet_gdf_10')
tweet_gdf_11 = assign_ct(tweet_gdf_11, 'tweet_gdf_11')
tweet_gdf_12 = assign_ct(tweet_gdf_12, 'tweet_gdf_12')

In [ ]:
tweet_gdf_01 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_01.pickle', 'rb'))
tweet_gdf_02 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_02.pickle', 'rb'))
tweet_gdf_03 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_03.pickle', 'rb'))
tweet_gdf_04 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_04.pickle', 'rb'))
tweet_gdf_05 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_05.pickle', 'rb'))
tweet_gdf_06 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_06.pickle', 'rb'))
tweet_gdf_07= pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_07.pickle', 'rb'))
tweet_gdf_08 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_08.pickle', 'rb'))
tweet_gdf_09 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_09.pickle', 'rb'))
tweet_gdf_10 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_10.pickle', 'rb'))
tweet_gdf_11 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_11.pickle', 'rb'))
tweet_gdf_12 = pickle.load(open(f'{data_dir}{sep}pickle{sep}tweet_gdf_12.pickle', 'rb'))

In [ ]:
i = 0
tweet_gdf_list = [tweet_gdf_01, tweet_gdf_02, tweet_gdf_03, tweet_gdf_04, tweet_gdf_05, tweet_gdf_06, tweet_gdf_07,
          tweet_gdf_08, tweet_gdf_09, tweet_gdf_10, tweet_gdf_11, tweet_gdf_12]
for df in tweet_gdf_list:
    i += len(df)
print(i,'tweets in total')

In [ ]:
# Create heat map for tweet counts per census tract over time
def collapse_append_df(df, df_to_append, collapse_function = "count"):
    ''' This function will collapse df by census tract/year and append it to df_to_append'''
    # Get the year and add it as a column
    df['year'] = pd.DatetimeIndex(df['date']).year
    if collapse_function == 'count':
        collapsed_df = df[['ym', 'LocationCT', 'year']].groupby(["LocationCT", "year"]) \
        .agg('count').rename(columns={'ym': 'count'}).reset_index(level=['LocationCT', 'year'])
    return df_to_append.append(collapsed_df)

In [ ]:
collapsed_count = pd.DataFrame()
# collapse each of the gdfs
for df in tweet_gdf_list:
    collapsed_count = collapse_append_df(df, collapsed_count)
# Add together the months of each year
collapsed_count = collapsed_count.groupby(['LocationCT', 'year']) \
.agg('sum').reset_index(level=['LocationCT', 'year'])

# Merge geometries back in using projected shapefile. Use a right merge to ensure all census tracts are 
# included (b/c not all census tracts are in the collapsed count for every year)
to_merge = nycct_proj[['geometry', 'fips_code']].rename(columns={'fips_code': 'LocationCT'})
collapsed_count_gdf = gp.GeoDataFrame(collapsed_count.merge(to_merge, how='right', on='LocationCT'))

In [ ]:
# Create heat map over time using collapsed_count_gdf
def create_heat_map(df, year, title, save_as):
    df[df['year'] == year].plot(column='count', legend=True)
    plt.title(f"{title}")
    plt.savefig(f"{cwd}{sep}figures{sep}{save_as}.jpg")
    plt.show()
    
for year in [2011,2012,2013]:
    year_str = str(year)
    create_heat_map(collapsed_count_gdf, year, f'Count of Geotagged Tweets by Census Tract in {year_str}',
                   f'geotagged_tweets_ct_{year_str}')

In [ ]:
collapsed_count_gdf['count'].describe()

#### Import and preprocess other data

##### 311 data

In [ ]:
# Bring in crime and 311 data, which uses the Socrata API in NYC Open Data
# Create a function that uses the Socrata API, which is written in SoQL, a SQL-like language, to query data
from sodapy import Socrata

def socrata_API_df(source_domain, dataset_id, select_string, where_string, limit=1000):
    '''
    Inputs: 
    source_domain: This tells Socrata the source of the dataset you're querying
    dataset_id: This is the unique id of the dataset
    select_string: This string tells Socrata which variables you are selecting from the dataset
    where_string: This string is equivalent to the "where" command in SQL
    limit = This tells Socrata how many results to query. The default is 1000 b/c Socrata automatically sets it to 1000

    Outputs a dataframe with with the queried results
    '''
    keyFile = open(f'{cwd}{sep}tokens{sep}socrata_apikey.txt', 'r')
    token = keyFile.readline() #api token imported from txt file
    
    client = Socrata(source_domain, token)
    # Change timeout var to arbitrarily large # of seconds so it doesn't time out
    client.timeout = 50
    results = client.get(dataset_id, limit = limit, select = select_string, where = where_string)
    df = pd.DataFrame.from_records(results)
    return df


In [ ]:
# Pull in 311 and Null Data 
# 2007, 2008, & 2009 are separate; 2010-on are in a single file. 
# The only thing that changes between 2007-09 is the dataset ID, & the id + where string for 2010-on
# so write a function that calls upon the 311 socrata API data
# complaint type string -- separated for ease of understanding. Complaint types drawn from literature
complaint_type_str = "complaint_type = 'Noise - Street/Sidewalk' OR complaint_type = 'Noise - Residential' OR complaint_type = 'Noise - Vehicle' OR complaint_type = 'Street Condition' " \
                    "OR complaint_type = 'Homeless Encampment' OR complaint_type = 'Drinking' OR complaint_type = 'Noise' " \
                    "OR complaint_type = 'Noise - Park' OR complaint_type = 'Noise - House of Worship' OR complaint_type = 'HEATING' " \
                    "OR complaint_type = 'GENERAL CONSTRUCTION' OR complaint_type = 'CONSTRUCTION' OR complaint_type = 'Boilers' " \
                    "OR complaint_type = 'For Hire Vehicle Complaint' OR complaint_type = 'Bike Rack Condition' OR complaint_type = 'Illegal Parking' " \
                    "OR complaint_type = 'Building/Use' OR complaint_type = 'ELECTRIC' OR complaint_type = 'PLUMBING'"

def pull_311(dataset_id, where_string = f'latitude IS NOT NULL AND ({complaint_type_str})'):
    return socrata_API_df(source_domain = "data.cityofnewyork.us", dataset_id = dataset_id, \
                         select_string = 'unique_key, created_date, complaint_type, date_extract_y(created_date) as year, date_extract_m(created_date) as month, descriptor, latitude, longitude', \
                         where_string = where_string,
                         limit = 4000000)

# 2007-2013
nyc_311_07 = pull_311("aiww-p3af")
nyc_311_08 = pull_311('uzcy-9puk')
nyc_311_09 = pull_311('3rfa-3xsf')
nyc_311_10_13 = pull_311('erm2-nwe9', \
                where_string = f'({complaint_type_str}) AND latitude IS NOT NULL AND (year = 2010 OR year = 2011 OR year = 2012 OR year = 2013)')

# Combine all four
nyc_311 = nyc_311_07.append(nyc_311_08).append(nyc_311_09).append(nyc_311_10_13).reset_index(drop=True)


In [ ]:
nyc_311.complaint_type.unique()

In [ ]:
nyc_311_pickle = open(f"{data_dir}{sep}pickle{sep}nyc_311.pickle", "wb")
pickle.dump(nyc_311, nyc_311_pickle)

In [ ]:
# Turn nyc 311 into a geodataframe
nyc_311 = pickle.load(open(f"{data_dir}{sep}pickle{sep}nyc_311.pickle", "rb"))
nyc_311_gdf = gp.GeoDataFrame(nyc_311, geometry=gp.points_from_xy(nyc_311.longitude, nyc_311.latitude))


In [ ]:
round_lat_long(nyc_311_gdf)

In [ ]:
# Run through the assign ct function
nyc_311_gdf = assign_ct(nyc_311_gdf, 'nyc_311_gdf', count_hundred=True)

In [ ]:
# Collapse 311 count by CT & year
nyc_311_collapsed = nyc_311_gdf[['descriptor','LocationCT', 'year']].groupby(["LocationCT", "year"]) \
        .agg('count').rename(columns={'descriptor': 'count'}).reset_index(level=['LocationCT', 'year'])

# Merge geometries back in using projected shapefile. Use a right merge to ensure all census tracts are 
# included (b/c not all census tracts are in the collapsed count for every year)
to_merge = nycct_proj[['geometry', 'fips_code']].rename(columns={'fips_code': 'LocationCT'})
collapsed_311_count = gp.GeoDataFrame(nyc_311_collapsed.merge(to_merge, how='right', on='LocationCT'))
collapsed_311_count

In [ ]:
collapsed_311_count['count'].describe()

In [ ]:
# Create heat maps using create_heat_map function from earlier
for year in [2010,2011,2012,2013]:
    year_str = str(year)
    create_heat_map(collapsed_311_count, year_str, f'Count of 311 Calls by Census Tract in {year_str}',
                   f'311_calls_ct_{year_str}')


##### crime data

In [ ]:
# Pull in NYC historical crime data (also uses Socrata data)
select_string = 'cmplnt_num, cmplnt_fr_dt AS date, date_extract_y(cmplnt_fr_dt) AS year,' \
    'date_extract_m(cmplnt_fr_dt) AS month,  pd_cd AS class, pd_desc, law_cat_cd AS level, crm_atpt_cptd_cd AS completed, latitude, longitude'
where_string = 'latitude IS NOT NULL AND (year = 2006 OR year = 2007 OR year = 2008 OR year = 2009 OR year = 2010 OR year = 2011 OR year = 2012 OR year = 2013)'
nyc_crime = socrata_API_df(source_domain = "data.cityofnewyork.us", dataset_id = 'qgea-i56i', \
                           select_string = select_string, where_string = where_string, limit = 4000000)


In [ ]:
# Pickle crime dataset
# pickle.dump(nyc_crime, open(f"{data_dir}{sep}pickle{sep}nyc_crime.pickle", "wb"))

In [ ]:
nyc_crime = pickle.load(open(f"{data_dir}{sep}pickle{sep}nyc_crime.pickle", "rb"))
nyc_crime_gdf = gp.GeoDataFrame(nyc_crime, geometry=gp.points_from_xy(nyc_crime.longitude, nyc_crime.latitude))

# Cut lat/long to 6 digits
round_lat_long(nyc_crime_gdf)

In [ ]:
# Assign CT
nyc_crime_gdf = assign_ct(nyc_crime_gdf, 'nyc_crime_gdf', count_hundred=True)

In [ ]:
# Collapse crime count by CT & year
nyc_crime_collapsed = nyc_crime_gdf[['cmplnt_num','LocationCT', 'year']].groupby(["LocationCT", "year"]) \
        .agg('count').rename(columns={'cmplnt_num': 'count'}).reset_index(level=['LocationCT', 'year'])

# Merge geometries back in using projected shapefile. 
to_merge = nycct_proj[['geometry', 'fips_code']].rename(columns={'fips_code': 'LocationCT'})
collapsed_crime_count = gp.GeoDataFrame(nyc_crime_collapsed.merge(to_merge, on='LocationCT'))

In [ ]:
# Make heat maps
for year in [2010,2011,2012,2013]:
    year_str = str(year)
    create_heat_map(collapsed_crime_count, year_str, f'Crime Count by Census Tract in {year_str}',
                   f'total_crime_ct_{year_str}')


##### HUD dataset

In [ ]:
# Bring in HUD vacant addresses data
# Create list of the excel files that will need to be loaded in 
# Glob.glob creates a list of all the files that end in .xlsx in the directory of HUD vacant data
# The rest of the command filters out jsons that end in 00000.json since those represent meta counts and not actual tweets
hud_list = [j for j in glob.glob(f'{data_dir}{sep}hud_vacant_data{sep}*.csv')]

In [ ]:
hud_df = pd.DataFrame()
for file in hud_list:
    temp_file = pd.read_csv(file, sep = None, engine='python')
#   Using title of the file, create a column for the year & month/quarter
    temp_file['year'] = ["20"+file[32:34] for i in range(temp_file.shape[0])]
    temp_file['month'] = [file[28:30] for i in range(temp_file.shape[0])]
    hud_df = hud_df.append(temp_file).reset_index(drop=True)


In [ ]:
# Create a FIPS code variable that is equal to the string of geoid 
# (note that b/c it's in integers, we need to re-add leading zero for states with fips codes < 10) 
hud_df['fips_code'] = hud_df["GEOID"].apply(lambda x: ("0" + str(x))[-11:])

# Create file with only NY 
ny_hud = hud_df[hud_df['fips_code'].apply(lambda x: x[0:2] == "36")].reset_index(drop=True) 

# Pickle ny hug file
nyc_hud_pickle = open(f"{data_dir}{sep}pickle{sep}nyc_hud.pickle", "wb")
pickle.dump(ny_hud, nyc_hud_pickle)

In [ ]:
# Unpickle ny hud
ny_hud = pickle.load(open(f"{data_dir}{sep}pickle{sep}nyc_hud.pickle", "rb"))

##### Businesses 

In [ ]:
# Bring in NYC businesses data. Some businesses have lat/long but not census tract and some vice versa
select_string = 'license_nbr, license_type, license_status, date_extract_y(license_creation_date) as year, date_extract_m(license_creation_date) as month,' \
                'industry, business_name, longitude, latitude'
where_string = 'latitude IS NOT NULL AND (year = 2011 OR year = 2012 OR year = 2013)'
nyc_bus = socrata_API_df("data.cityofnewyork.us", "w7w3-xahh", select_string = select_string, 
               where_string = where_string, limit = 50000)

pickle.dump(nyc_bus, open(f"{data_dir}{sep}pickle{sep}nyc_businesses.pickle", "wb"))

In [ ]:
# Unpickle 
nyc_bus = pickle.load(open(f"{data_dir}{sep}pickle{sep}nyc_businesses.pickle", "rb"))

nyc_bus_gdf = gp.GeoDataFrame(nyc_bus, geometry=gp.points_from_xy(nyc_bus.longitude, nyc_bus.latitude))
# Cut lat/long to 6 digits
round_lat_long(nyc_bus_gdf)

In [ ]:
nyc_bus_gdf = assign_ct(nyc_bus_gdf, 'nyc_bus_gdf')